In [23]:
MAX_EPOCHS = 2000
PATIENCE = 100
normalize = False

In [24]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import DateTimeHelper as help

In [25]:
df = help.getDT("Twisted bow", "5m")
df2 = df.copy()

layer = tf.keras.layers.Normalization(invert=True)
layer.adapt(df)

df = help.addAverage(df)

if(normalize):
    df = tf.keras.utils.normalize(df)

print(np.any(np.isnan(df)))

False


In [26]:
mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [27]:
column_indices = {name: i for i, name in enumerate(df.columns)}

n = len(df)
print(n)
print(len(df2))
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]
og_test_df = df2[int(n*0.9):]

num_features = df.shape[1]

365
365


In [28]:
train_mean = train_df.mean()
train_std = train_df.std()

# train_df = (train_df - train_mean) / train_std
# val_df = (val_df - train_mean) / train_std
# test_df = (test_df - train_mean) / train_std

In [29]:
# df_std = (df - train_mean) / train_std
# df_std = df_std.melt(var_name='Column', value_name='Normalized')

In [30]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    # self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [31]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [32]:
def plot(self, model=None, plot_col='average', max_subplots=5):
  inputs, labels = self.example
  plt.figure(figsize=(12, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(max_n, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]')
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [33]:
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.utils.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

In [34]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [35]:
single_step_window = WindowGenerator(
    input_width=30, label_width=1, shift=1,
    label_columns=['average'])
single_step_window

Total window size: 31
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
Label indices: [30]
Label column name(s): ['average']

In [36]:
# linear
linear = tf.keras.Sequential([
    tf.keras.layers.Dense(units=1)
])

In [37]:
print('Input shape:', single_step_window.example[0].shape)
print('Output shape:', linear(single_step_window.example[0]).shape)

Input shape: (32, 30, 5)
Output shape: (32, 30, 1)


In [38]:
def compile_and_fit(model, window, patience=PATIENCE):
  early_stopping1 = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                                    patience=patience,
                                                    start_from_epoch=MAX_EPOCHS/5,
                                                    mode='min')
  early_stopping2 = tf.keras.callbacks.EarlyStopping(monitor='mean_absolute_error',
                                                    patience=patience,
                                                    start_from_epoch=MAX_EPOCHS/5,
                                                    mode='min')
  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.legacy.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping1, early_stopping2])
  return history

In [39]:
history = compile_and_fit(linear, single_step_window)

Epoch 1/2000
8/8 [==============================] - 0s 25ms/step - loss: 6839550563487580160.0000 - mean_absolute_error: 2615235584.0000 - val_loss: 6765988287787040768.0000 - val_mean_absolute_error: 2601148416.0000
Epoch 2/2000
8/8 [==============================] - 0s 10ms/step - loss: 6667010800810459136.0000 - mean_absolute_error: 2582038016.0000 - val_loss: 6594152762307510272.0000 - val_mean_absolute_error: 2567905280.0000
Epoch 3/2000
8/8 [==============================] - 0s 13ms/step - loss: 6497096122144653312.0000 - mean_absolute_error: 2548922880.0000 - val_loss: 6425109446606716928.0000 - val_mean_absolute_error: 2534777344.0000
Epoch 4/2000
8/8 [==============================] - 0s 12ms/step - loss: 6330018733234323456.0000 - mean_absolute_error: 2515935744.0000 - val_loss: 6259009523533479936.0000 - val_mean_absolute_error: 2501798400.0000
Epoch 5/2000
8/8 [==============================] - 0s 11ms/step - loss: 6165921570591080448.0000 - mean_absolute_error: 2483111936.

In [40]:
# wide_window.plot(linear)
y_pred = linear.predict(test_df)
for i in range(len(y_pred)):

    if normalize:
        print("" + str(np.array(layer(y_pred[i]))[0][4]) + " vs " + str(np.array(layer(test_df['average'][i]))[0][4]) + " vs " + str(og_test_df['average'][i]))
    else:
        print("" + str(y_pred[i][0]) + " vs " + str(test_df['average'][i]))

print(test_df.shape)
print(og_test_df.shape)


2/2 [==============================] - 0s 5ms/step
1386270700.0 vs 1386000000.0
1385370500.0 vs 1385100000.0
1387371500.0 vs 1387141333.3333333
1391605100.0 vs 1391333333.0
1385125000.0 vs 1384854504.0
1384937500.0 vs 1384666999.0
1390271500.0 vs 1390000000.0
1388604500.0 vs 1388333333.0
1387116000.0 vs 1386845153.0
1388916400.0 vs 1388645153.0
1384603600.0 vs 1384333333.0
1384721400.0 vs 1384451000.0
1384777300.0 vs 1384507305.3333333
1391271800.0 vs 1391000000.0
1386210700.0 vs 1388848857.0
1387771100.0 vs 1387500000.0
1391159700.0 vs 1390888000.0
1384871300.0 vs 1384601000.0
1391271800.0 vs 1391000000.0
1390271500.0 vs 1390000000.0
1385047400.0 vs 1384777000.0
1384906800.0 vs 1384636350.0
1386042600.0 vs 1389209889.0
1390971600.0 vs 1390700000.0
1386270700.0 vs 1386000000.0
1385493000.0 vs 1385222222.0
1390271500.0 vs 1390000000.0
1386770800.0 vs 1386500000.0
1385286000.0 vs 1385015555.0
1390159400.0 vs 1389888000.0
1391271800.0 vs 1391000000.0
1386970900.0 vs 1387955500.0
138528600

In [41]:
test_df

,avgHighPrice,avgLowPrice,highPriceVolume,lowPriceVolume,average
timestamp,,,,,
2023-04-06 13:10:00,1.386000e+09,1.386000e+09,0,2,1.386000e+09
2023-04-06 13:20:00,1.385100e+09,1.385100e+09,0,1,1.385100e+09
2023-04-06 13:35:00,1.390580e+09,1.385422e+09,1,2,1.387141e+09
2023-04-06 13:40:00,1.391333e+09,1.391333e+09,1,0,1.391333e+09
2023-04-06 13:45:00,1.384855e+09,1.384855e+09,0,2,1.384855e+09
2023-04-06 13:50:00,1.384667e+09,1.384667e+09,0,1,1.384667e+09
2023-04-06 13:55:00,1.390000e+09,1.390000e+09,0,2,1.390000e+09
2023-04-06 14:00:00,1.388333e+09,1.388333e+09,0,3,1.388333e+09
2023-04-06 14:15:00,1.386845e+09,1.386845e+09,3,0,1.386845e+09
